In [1]:
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import random as r
from matplotlib.patches import Ellipse, Arc
import os
import glob
from sympy import Point, Line, Segment

from tqdm import tqdm

import math
from time import gmtime, strftime

In [2]:
class MathDot:
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __str__(self):
        return "Dot - (" + str(self.x) + ", " + str(self.y) + ")\n"
        
    def dX(dotOne, dotTwo):
        return dotTwo.x - dotOne.x
    
    def dY(dotOne, dotTwo):
        return dotTwo.y - dotOne.y
    
    def length(dotOne, dotTwo):
        return math.sqrt(pow(MathDot.dY(dotOne, dotTwo), 2) + pow(MathDot.dX(dotOne, dotTwo), 2))
    
    def makeVector(dotOne, dotTwo):
        return MathDot(dotTwo.x - dotOne.x, dotTwo.y - dotOne.y)
    
    def vectorProduct(dotOne, dotTwo):
        return dotOne.x * dotTwo.y - dotOne.y * dotTwo.x
    
class Dot(MathDot):
    
    def __init__(self, x, y, parentBranch):
        self.isBusy = False
        super().__init__(x, y)
        self.parentBranch = parentBranch
        self.childBranch = None
        
    def addChildBranch(self, childBranch):
        self.childBranch = childBranch

In [3]:
class Direction:
    
    def __init__(self, degree):
        self.degree = degree
        self.sin = round(math.sin(math.radians(degree)),3)
        self.cos = round(math.cos(math.radians(degree)),3)
        
    def __add__(self, other):
        if isinstance(other, Direction):
            return Direction(self.degree + other.degree)
        else:
            return Direction(self.degree + other)
        
    def __sub__(self, other):
        if isinstance(other, Direction):
            return Direction(self.degree - other.degree)
        else:
            return Direction(self.degree - other)

In [4]:
class Branch:
    
    def __init__(self, xStart, yStart, direction, fatherBranch = None):
        self.startCoordinates = (xStart, yStart)
        self.endCoordinates = [xStart, yStart]
        self.direction = direction
        self.dots = [Dot(xStart, yStart, self)]
        self.childBranches = []
        self.fatherBranch = fatherBranch
        self.canGrow = True
        self.intersectedBranches = [[]]
        self.canBeRemoved = True
        
    def __str__(self):
        string = "Branch:\nFirst dot - (" + str(self.dots[0].x) + ", " + str(self.dots[0].y) + ")\n" + \
        "Last dot - (" + str(self.dots[-1].x) + ", " + str(self.dots[-1].y) + ")\nDirection - " + str(self.direction.degree) + "\n"
        return string
        
    def addDot(self):
        self.endCoordinates[0] += self.direction.cos * CONSTS.MIN_INCREASE_LENGTH
        self.endCoordinates[1] += self.direction.sin * CONSTS.MIN_INCREASE_LENGTH
        newDot = Dot(self.endCoordinates[0], self.endCoordinates[1], self)
        self.dots.append(newDot)
        
        self.intersectedBranches.append([])
        self.canBeRemoved = False
        
        return newDot
    
    def removeDot(self):
        for branch in self.intersectedBranches[-1]:
            branch.canGrow = True
        self.intersectedBranches.pop(-1)
        del self.dots[-1]
        self.endCoordinates[0] = self.dots[-1].x
        self.endCoordinates[1] = self.dots[-1].y
        self.canGrow = True
        
    def addChildBranch(self, dot, coef):
        coef = 1 if coef >= 0 else -1
        newDirection = Direction(self.direction.degree + coef * CONSTS.NEW_BRANCH_ANGLE_DEGREE)
        thisDot = dot
        nextDot = self.dots[self.dots.index(dot) + 1]
        thisDot.isBusy = True
        nextDot.isBusy = True
        newBranchXStart = (thisDot.x + nextDot.x) / 2
        newBranchYStart = (thisDot.y + nextDot.y) / 2
        newBranch = Branch(newBranchXStart, newBranchYStart, newDirection, self)
        thisDot.childBranch = newBranch
        nextDot.childBranch = newBranch
        self.childBranches.append(newBranch)
        return newBranch
        
    def getFreeDots(self):
        res = []
        if len(self.dots) > 2:
            for i in range(1, len(self.dots) - 1):
                if self.dots[i].isBusy == False and self.dots[i + 1].isBusy == False:
                    res.append(self.dots[i])
        return res
    
    def addIntersection(self, intersectionDot, branch):
        fromStartToIntersectionLength = MathDot.length(self.dots[0], intersectionDot)
        intersectedDotIndex = len(self.dots) - 1 - (self.length() - fromStartToIntersectionLength) // CONSTS.MIN_INCREASE_LENGTH
        intersectedDotIndex = int(intersectedDotIndex)
        if intersectedDotIndex == len(self.dots):
            intersectedDotIndex -= 1
        try:
            self.intersectedBranches[intersectedDotIndex].append(branch)        
        except:
            Error.value = (self, intersectionDot, branch)
            print(intersectionDot, intersectedDotIndex, len(self.intersectedBranches), self, branch)
            raise Exception("seva")
        
    def dX(self):
        return self.endCoordinates[0] - self.startCoordinates[0]
    
    def dY(self):
        return self.endCoordinates[1] - self.startCoordinates[1];
    
    def length(self):
        return math.sqrt(pow(self.dX(), 2) + pow(self.dY(), 2))
    
    def getXCoordinates(self):
        return [self.startCoordinates[0], self.endCoordinates[0]]

    def getYCoordinates(self):
        return [self.startCoordinates[1], self.endCoordinates[1]]

In [5]:
class Tree:
    
    def __init__(self, root):
        self.root = root
            
    def allBranchesThatCanGrow(self):
        res = []
        for branch in self.getAllBranches():
            if branch.canGrow and Checker.checkBranchCanGrow(tree, branch):
                res.append(branch)
        return res
    
    def allBranchesThatCanBreak(self):
        res = []
        for branch in self.getAllBranches():
            if Checker.checkBranchCanBreak(branch):
                res.append(branch)
        return res
    
    def getAllBranches(self):
        res = []
        self.__recGetAllBranches(self.root, res)
        return res
        
    def __recGetAllBranches(self, branch, res):
        res.append(branch)
        for b in branch.childBranches:
            self.__recGetAllBranches(b, res)
    
    def getFreeDots(self):
        res = []
        for branch in self.getAllBranches():
            for dot in branch.getFreeDots():
                res.append(dot)
        return res
    
    def removeBranch(self, branch):
        for intersection in branch.intersectedBranches:
            for intersectedBranch in intersection:
                intersectedBranch.canGrow = True
        branch.fatherBranch.childBranches.remove(branch)
        for dot in branch.fatherBranch.dots:
            if dot.childBranch == branch:
                dot.childBranch = None
                dot.IsBusy = False
    

In [68]:
class Cell:

    def __init__(self, initCoords : list, initRad : int):
        self.centre = Dot(initCoords[0], initCoords[1], None)
        self.startRadius = initRad
        self.radius = initRad
        self.leanedBranches = []
        
    def grow(self):
        for lBranch in self.leanedBranches:
            lBranch.canGrow = True
        self.leanedBranches.clear()
        self.radius += CONSTS.CELL_RADIUS_PLUS
    
    def addLeanedBranch(self, branch):
        self.leanedBranches.append(branch)
        branch.canGrow = False

In [69]:
class Plot:
    
    def __init__(self, x_limits : list, y_limits : list):
        self.x_limits = x_limits
        self.y_limits = y_limits
        
    def drawPlot(self, tree : Tree, cell : Cell, index : int):
        self.fig, self.ax = plt.subplots()
        self.ax.set_xlim(self.x_limits)
        self.ax.set_ylim(self.y_limits)
        self.__drawTree(tree)
        self.__drawCell(cell)
        self.__drawText(cell, tree)
        self.__savePlot(index)
    
    def __savePlot(self, index : int):
        self.fig.set_size_inches(10,20)
        plt.savefig("img/scr_" + str(index) + ".png")
        plt.close('all')
    
    def __drawTree(self, tree : Tree):
        for branch in tree.getAllBranches():
            self.ax.plot(branch.getXCoordinates(), branch.getYCoordinates(), c = 'r')
            
    def __drawCell(self, cell : Cell):
        cellWall = Arc(xy = [cell.centre.x, cell.centre.y], width = cell.radius * 2, height = cell.radius * 2,theta1 = -90, theta2 = 90, color = "g")
        cellAttachArea = Arc(xy = [cell.centre.x, cell.centre.y], width = (cell.radius - CONSTS.DISTANCE_TO_ATTACH) * 2, height = (cell.radius - CONSTS.DISTANCE_TO_ATTACH) * 2, theta1 = -90, theta2 = 90, color = "g", ls = '--')
        cellGrowArea = Arc(xy = [cell.centre.x, cell.centre.y], width = (cell.radius - CONSTS.DISTANCE_TO_GROW) * 2, height = (cell.radius - CONSTS.DISTANCE_TO_GROW) * 2, theta1 = -90, theta2 = 90, color = "g", ls = '-')
        
        self.ax.add_patch(cellWall) 
        self.ax.add_patch(cellAttachArea) 
        self.ax.add_patch(cellGrowArea)
        
        leanedDots = [x.dots[-1] for x in cell.leanedBranches]
        circles = [plt.Circle((dot.x, dot.y), 0.3, color = 'b') for dot in leanedDots]
        for circle in circles:
            self.ax.add_patch(circle)
    
    def __drawText(self, cell : Cell, tree : Tree):
        text = ""
        text += "\nn: " + str(CONSTS.N_BRANCHES_TO_PUSH_SELL)
        text += "\nKb: " + str(CONSTS.Kb)
        text += "\nH: " + str(CONSTS.DISTANCE_TO_GROW)
        text += "\nD: " + str(CONSTS.DISTANCE_TO_ATTACH)
        text += "\navgV: " + str((cell.radius - cell.startRadius) * 5.5e-9 / 150)
        self.ax.annotate(text, xy=(1000, 500), color = 'r')

In [8]:
class Checker:
    
    def checkBranchCanGrow(tree, branch):   
        if MathDot.length(cell.centre, branch.dots[-1]) > cell.radius or MathDot.length(cell.centre, branch.dots[-1]) < cell.radius - CONSTS.DISTANCE_TO_GROW:
            return False
        for anotherBranch in tree.getAllBranches():
            if anotherBranch != branch and anotherBranch != branch.fatherBranch and anotherBranch not in branch.childBranches:
                res = Checker.checkGrowBranchCrossAnother(branch, anotherBranch)
                if res != False:
                    branch.canGrow = False
                    if res != True:
                        anotherBranch.addIntersection(res, branch)
                    return False
        return True
    
    def checkBranchCanBreak(branch):
        return branch.dots[-1].isBusy == False and len(branch.dots) > 2
    
    def checkGrowBranchCrossAnother(branchToGrow, branchTwo):
        
        
        
        a = branchToGrow.dots[0]
        lastDot = branchToGrow.dots[-1]
        b = MathDot(lastDot.x + branchToGrow.direction.cos * CONSTS.MIN_INCREASE_LENGTH, lastDot.y + branchToGrow.direction.sin * CONSTS.MIN_INCREASE_LENGTH)
        c = branchTwo.dots[0]
        d = branchTwo.dots[-1]
        
        return Checker._intersection(a,b,c,d)
        
    def _intersection(a,b,c,d):
        x1 = a.x
        y1 = a.y
        x2 = b.x
        y2 = b.y
        x3 = c.x
        y3 = c.y
        x4 = d.x
        y4 = d.y

        A1 = y1 - y2
        A2 = y3 - y4
        B1 = x2 - x1
        B2 = x4 - x3
        C1 = x1 * y2 - x2 * y1
        C2 = x3 * y4 - x4 * y3
        
        if abs(c.x - d.x) <= 1e-10:
            return Checker._dotIsOnSegment(c, a, b)

        if abs(A1) <= 1e-10 and abs(A2) <= 1e-10:
            if C1 / B1 == C2 / B2:
                m = max(x1, x2, x3, x4)
                if m == x1 or m == x2:
                    m = min(x1, x2)
                    return Checker._dotIsOnSegment(a if m == x1 else b, c, d)
                else:
                    m = min(x3, x4)
                    return Checker._dotIsOnSegment(c if m == x3 else d, a, b)
            else:
                return False

        if abs(A1) <= 1e-10:
            y = -C1 / B1
            x = -C2 / A2 + -B2 / A2 * y
            if Checker._dotIsInTwoSegments(MathDot(x,y), a, b, c, d):
                return MathDot(x,y)
            else:
                return False

        if abs(A2) <= 1e-10:
            y = -C2 / B2
            x = -C1 / A1 + -B1 / A1 * y
            if Checker._dotIsInTwoSegments(MathDot(x,y), a, b, c, d):
                return MathDot(x,y)
            else:
                return False

        value = B1 / A1 - B2 / A2
        if abs(value) <= 1e-10:
            return False

        y = (C2 / A2 - C1 / A1) / value
        x = -C1 / A1 + -B1 / A1 * y
        if Checker._dotIsInTwoSegments(MathDot(x,y), a, b, c, d):
            return MathDot(x,y)
        else:
            return False
    
    def _dotIsInTwoSegments(dot, a, b, c, d):
        return Checker._dotIsOnSegment(dot, a, b) and Checker._dotIsOnSegment(dot, c, d)

    def _dotIsOnSegment(dot, a, b):
        #print(dot, a, b)
        x = dot.x
        y = dot.y
        x1 = a.x
        y1 = a.y
        x2 = b.x
        y2 = b.y
    
        return abs((x - x1) * (y2 - y1) - (y - y1) * (x2 - x1)) <= 1e-10 and (x1 <= x <= x2 or x2 <= x <= x1)
        
    def checkBranchCanHaveChildBranches(branch, coef):
        coef = 1 if coef >= 0 else -1
        newAngle = branch.direction.degree + coef * CONSTS.NEW_BRANCH_ANGLE_DEGREE
        if newAngle > CONSTS.MAX_BRANCH_ANGLE or newAngle < CONSTS.MIN_BRANCH_ANGLE:
            return False
        return True
    
    
    def checkDotCanJoin(cell : Cell, dot : Dot):
        if MathDot.length(cell.centre, dot) < cell.radius and MathDot.length(cell.centre, dot) > cell.radius - CONSTS.DISTANCE_TO_ATTACH:
            return True
        return False
    
    def checkDotOutsideCell(cell : Cell, dot : Dot):
        if MathDot.length(cell.centre, dot) > cell.radius:
            return True
        return False
    
    def checkCellNeedToGrow(cell : Cell):
        return len(cell.leanedBranches) >= CONSTS.N_BRANCHES_TO_PUSH_SELL

In [9]:
class Error:
    value = 0

In [10]:
class Randomizer:

    def __init__(self, tree : Tree, cell : Cell):
        self.tree = tree
        self.cell = cell
    
    def oneTick(self):
        if Checker.checkCellNeedToGrow(self.cell):
            self.cell.grow()
        self.grow()
        self.breakBranches()
        self.addChildBranches()
        self.removeNewBranches()

                
    def grow(self):
        for branch in self.tree.allBranchesThatCanGrow():
            if r.random() < CONSTS.ADD_DOT_PROB:
                newDot = branch.addDot()
                if (Checker.checkDotOutsideCell(self.cell, newDot)):
                    self.cell.addLeanedBranch(newDot.parentBranch)
    
                
    def addChildBranches(self):
        allDots = self.tree.getFreeDots()
        for dot in allDots:
            if dot.isBusy == False and Checker.checkDotCanJoin(self.cell, dot) and r.random() < CONSTS.CREATE_BRANCH_PROB:
                if Checker.checkBranchCanHaveChildBranches(dot.parentBranch, 1) == False:
                    coef = -1
                elif Checker.checkBranchCanHaveChildBranches(dot.parentBranch, -1) == False:
                    coef = 1
                else:
                    coef = 1 if r.random() >= 0.5 else -1
                dot.parentBranch.addChildBranch(dot, coef)
                
    def breakBranches(self):
        for branch in self.tree.allBranchesThatCanBreak():
            if r.random() < CONSTS.REMOVE_DOT_PROB:
                branch.removeDot()
                if branch in self.cell.leanedBranches:
                    self.cell.leanedBranches.remove(branch)
                    
    def removeNewBranches(self):
        allBranches = self.tree.getAllBranches()
        allBranches.remove(self.tree.root)
        branchesThatCanBeRemoved = [x for x in allBranches if x.canBeRemoved == True]
        for branch in branchesThatCanBeRemoved:
            if r.random() < CONSTS.REMOVE_BRANCH_PROB:
                tree.removeBranch(branch)

In [91]:
class CONSTS: 
    DT = 0.1
    K1G = 10
    K2 = 1.4
    K3ARP = 10
    X_DENSITY = 9.7e-6
    Kb = 20000
    ADD_DOT_PROB = 1-1/(math.exp(DT*K1G))
    REMOVE_DOT_PROB = 1-1/(math.exp(DT*K2))
    CREATE_BRANCH_PROB = 1-1/(math.exp(DT*Kb*X_DENSITY))
    REMOVE_BRANCH_PROB = 0
    #ADD_DOT_PROB = 0.5
    #REMOVE_DOT_PROB = 0.2
    #CREATE_BRANCH_PROB = 0.9
    #REMOVE_BRANCH_PROB = 0.2
    N_BRANCHES_TO_PUSH_SELL = 10
    NEW_BRANCH_ANGLE_DEGREE = 70
    MIN_INCREASE_LENGTH = 1
    MAX_BRANCH_ANGLE = 90
    MIN_BRANCH_ANGLE = -90
    DISTANCE_TO_ATTACH = 10
    DISTANCE_TO_GROW = 10
    CELL_RADIUS_PLUS = 1

In [92]:
print("ADD_DOT_PROB" , CONSTS.ADD_DOT_PROB)
print("REMOVE_DOT_PROB" , CONSTS.REMOVE_DOT_PROB)
print("CREATE_BRANCH_PROB" , CONSTS.CREATE_BRANCH_PROB)

ADD_DOT_PROB 0.6321205588285577
REMOVE_DOT_PROB 0.13064176460119403
CREATE_BRANCH_PROB 0.019213031018206927


In [76]:
tree = Tree(Branch(0,0,Direction(0),None))
cell = Cell([0,0], 10)
randomizer = Randomizer(tree, cell)
plot = Plot([0, 1500], [-1000, 1000])
index = 0
for i in tqdm(range(1500)):
    randomizer.oneTick()
    index += 1
    #plot.drawPlot(tree, cell, index)
plot.drawPlot(tree, cell, strftime("%Y-%m-%d_%H_%M", gmtime()))

 56%|████████████████████████████████████████████▏                                  | 840/1500 [04:46<03:44,  2.93it/s]


KeyboardInterrupt: 

In [42]:
print(cell.radius)

282


In [77]:
plot = Plot([0, 1500], [-1000, 1000])
plot.drawPlot(tree, cell, strftime("%Y-%m-%d_%H_%M", gmtime()))

In [78]:
print(len(tree.getAllBranches()))

2012
